In [22]:
import pyspark
from pyspark.sql import SparkSession
from graphframes import GraphFrame

from gensim import corpora
import sys
import re

sys.path.append("../")  # Add "../" to utils folder path
from utils import globals

In [25]:
FILENAME_GL = globals.DATA_PATH + 'output_2_2.txt'
FILENAME_CORPUS = globals.DATA_PATH + 'output_1_3.txt'

In [20]:
DICTIONARY_PATH = globals.DATA_PATH + "dictionary.pkl"

## Read input file

In [26]:
# Test line cleaning method
with open(FILENAME_GL) as f:
    content = f.readlines()
    
a = content[0]

re.sub('[null\t\n\[\]\""]', '', a).replace(' ', '').split(',')

['438']

In [27]:
GL = []

with open(FILENAME_GL) as f:
    content_vertices = f.readlines()

for line in content_vertices:
    clean_line = re.sub('[null\t\n\[\]\""]', '', line).replace(' ', '').split(',')
    GL.append(list(map(int, clean_line))) # Convert all strings in a list to int
    
print(GL[:5])

[[438], [1251], [119], [937], [881]]


Load dictionary file

In [23]:
dictionary = corpora.Dictionary.load(DICTIONARY_PATH)

Load corpus

In [59]:
with open(FILENAME_CORPUS) as f:
    content = f.readlines()
    
a = content[0]

re.sub('[null\t\n]', '', a)

new_content = [x.strip() for x in content] 
content0 = re.sub('[null\t\n]', '', new_content[0])
content0

'[0, [[0, 21], [1, 4], [2, 5], [3, 18], [4, 4], [5, 4], [6, 1], [7, 10], [8, 2], [9, 2], [10, 3], [11, 1], [12, 18], [13, 2], [14, 4], [15, 14], [16, 3], [18, 1], [19, 6], [20, 5], [21, 3], [23, 5], [24, 2], [27, 1], [28, 3], [29, 3], [30, 3], [31, 1], [32, 1], [34, 2], [35, 3], [36, 1], [37, 1], [38, 5], [41, 1], [42, 4], [43, 2], [44, 3], [45, 19], [46, 4], [47, 4], [48, 5], [49, 3], [50, 4], [52, 3], [54, 2], [55, 6], [56, 5], [57, 3], [58, 8], [59, 2], [61, 4], [62, 3], [63, 1], [64, 1], [65, 3], [67, 5], [68, 6], [69, 1], [70, 5], [71, 5], [72, 2], [73, 3], [74, 5], [76, 1], [79, 2], [80, 4], [81, 1], [82, 1], [91, 7], [92, 3], [94, 3], [95, 4], [97, 4], [98, 3], [99, 14], [100, 2], [101, 1], [103, 2], [105, 2], [106, 1], [107, 1], [110, 1], [114, 2], [116, 1], [117, 1], [120, 1], [121, 2], [124, 2], [125, 3], [126, 16], [127, 2], [128, 1], [129, 8], [130, 6], [131, 8], [133, 2], [134, 1], [135, 18], [136, 6], [137, 2], [138, 1], [139, 7], [140, 2], [141, 18], [142, 4], [143, 4], 

In [68]:
# Use re.sub. Just match all the chars upto "[[" then replace the matched chars with "[".
# And remove last 2 character of the string ("]]")
re.sub(r'^.*?\[\[', '[', content0)[:-2]

'[0, 21], [1, 4], [2, 5], [3, 18], [4, 4], [5, 4], [6, 1], [7, 10], [8, 2], [9, 2], [10, 3], [11, 1], [12, 18], [13, 2], [14, 4], [15, 14], [16, 3], [18, 1], [19, 6], [20, 5], [21, 3], [23, 5], [24, 2], [27, 1], [28, 3], [29, 3], [30, 3], [31, 1], [32, 1], [34, 2], [35, 3], [36, 1], [37, 1], [38, 5], [41, 1], [42, 4], [43, 2], [44, 3], [45, 19], [46, 4], [47, 4], [48, 5], [49, 3], [50, 4], [52, 3], [54, 2], [55, 6], [56, 5], [57, 3], [58, 8], [59, 2], [61, 4], [62, 3], [63, 1], [64, 1], [65, 3], [67, 5], [68, 6], [69, 1], [70, 5], [71, 5], [72, 2], [73, 3], [74, 5], [76, 1], [79, 2], [80, 4], [81, 1], [82, 1], [91, 7], [92, 3], [94, 3], [95, 4], [97, 4], [98, 3], [99, 14], [100, 2], [101, 1], [103, 2], [105, 2], [106, 1], [107, 1], [110, 1], [114, 2], [116, 1], [117, 1], [120, 1], [121, 2], [124, 2], [125, 3], [126, 16], [127, 2], [128, 1], [129, 8], [130, 6], [131, 8], [133, 2], [134, 1], [135, 18], [136, 6], [137, 2], [138, 1], [139, 7], [140, 2], [141, 18], [142, 4], [143, 4], [144,

## Compute distribution

In [ ]:
def compute_dist(dist, groups, seeds, only_seed=True):
    res = []
    if only_seed:
        print(seeds)
        for seednodes in seeds:
            tmp = dist[seednodes, :]
            print(tmp)
            if globals.GROUP_AGGREGATION == "MEAN":
                res += [np.mean(tmp, axis=0)]
            elif globals.GROUP_AGGREGATION == "MEDIAN":
                res += [np.median(tmp, axis=0)]
    else:
        for groupnodes in groups:
            tmp = dist[groupnodes, :]
            if globals.GROUP_AGGREGATION == "MEAN":
                res += [np.mean(tmp, axis=0)]

            elif globals.GROUP_AGGREGATION == "MEDIAN":
                res += [np.median(tmp, axis=0)]
    print("------------------")
    print(np.array(res))
    print("------------------")
    return np.array(res)

In [ ]:
def cluster_groups( group_dist ):
    if SCALING:
        scaler = preprocessing.StandardScaler()
        X_scaled = scaler.fit_transform(group_dist)
        print(X_scaled)
    else:
        X_scaled = group_dist

    if CLUSTERING_METHOD == 'KMEANS':
        # clustering by k-means
        kmeans = KMeans( n_clusters=NUM_OF_SPECIES, init='k-means++')
        y_grp_cl = kmeans.fit_predict( X_scaled )
        
    elif CLUSTERING_METHOD == 'SPECTRAL':
        spectral = SpectralClustering(n_clusters=NUM_OF_SPECIES, eigen_solver='arpack',
                                      affinity="nearest_neighbors")
        #spectral = SpectralClustering(n_clusters=NUM_OF_SPECIES, eigen_solver='arpack',
        #                              affinity="rbf")
        y_grp_cl = spectral.fit_predict( X_scaled )
    print(y_grp_cl)
    return y_grp_cl;

Run command for `compute_dist()`
```python
corpus_m = gensim.matutils.corpus2dense(corpus, len(dictionary.keys())).T
kmer_group_dist = compute_dist( corpus_m, GL, SL, only_seed=False)
z_kmer_grp_cl = cluster_groups( kmer_group_dist )
```

In [ ]:
corpus_m = gensim.matutils.corpus2dense(corpus, len(dictionary.keys())).T